<a href="https://colab.research.google.com/github/darkmochalover/DS_TermProject_AL2/blob/main/decision%20tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary

In [27]:
# Data Path
raw_data_path = "/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv"
# data_path = "data/preprocessed_mbti_data.csv"


audio_qualities = [
    'danceability_mean',
    'valence_mean',
    'energy_mean',
    'loudness_mean',
    'acousticness_mean',
    'instrumentalness_mean',
    'liveness_mean',
]

# 장조/단조 (Major/Minor)
all_tones = [
    'Cminor_count', 'CMajor_count', 'C#/Dbminor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 'Eminor_count', 'EMajor_count',
    'Fminor_count', 'FMajor_count', 'F#/Gbminor_count', 'GMajor_count',
    'G#/Abminor_count', 'G#/AbMajor_count', 'Aminor_count', 'AMajor_count',
    'A#/Bbminor_count', 'BMajor_count', 'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 'A#/BbMajor_count', 'F#/GbMajor_count', 'Bminor_count'
]

major_tones = [
    'CMajor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 
    'EMajor_count',
    'FMajor_count', 
    'GMajor_count', 'G#/AbMajor_count', 
    'AMajor_count', 'BMajor_count', 'A#/BbMajor_count', 
    'F#/GbMajor_count'
]
minor_tones = [
    'Cminor_count', 'C#/Dbminor_count', 
    'Eminor_count', 
    'Fminor_count', 'F#/Gbminor_count', 
    'G#/Abminor_count',  
    'Aminor_count', 'A#/Bbminor_count', 
    'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 
    'Bminor_count'
]


# 열 이름을 바꾸기 위해 리스트로 저장함
renamed_columns =  [
    'danceability',
    'valence',
    'energy',
    'loudness',
    'acousticness',
    'instrumentalness',
    'liveness'
]


Install WandB

In [28]:
!pip install -qU wandb

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

import itertools

import wandb
import random

Step 1: Import W&B and Login

In [30]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

In [31]:
encoder_list = ['LabelEncoder', 'OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote = ['True', 'False']

In [32]:
config = {
        'encoder' : 'LabelEncoder',
        'scaler' : 'StandardScaler',
        'test_size' : 0.3,
        "do_smote" : True
    }

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="MBTI_playlist",
    
    # track hyperparameters and run metadata
    config = config
)

config=wandb.config


wandb: Currently logged in as: akdl0108 (ds_2023_spring). Use `wandb login --relogin` to force relogin


Data Loading

In [33]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

raw_data_path = '/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv'
df = pd.read_csv(raw_data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Feature Engineering

In [34]:
# Subset only measures of centers
X = df.iloc[: , :22] # 처음 22개 열을 선택해서 저장
X = df[audio_qualities] # 오디오 품질과 해당 열이 있는 열을 선택해서 저장


categories = renamed_columns[:]
X.columns = renamed_columns


# 장조/단조의 개수의 합을 계산해서 저장 (C장조, D단조, .. 이렇게 따로 계산되는거 말고, 위에 지정된 list 이용해서 sum값 넣어줌)
X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')

<ipython-input-34-482ca4c8bc7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
<ipython-input-34-482ca4c8bc7c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')


## Encoding & Scaling Function

In [35]:
def Encoding(df, encoding_method):
    df = df.copy()

    if(encoding_method == 'LabelEncoder'):
        encoder = LabelEncoder()
        target = encoder.fit_transform(df[['mbti']])
        

    if(encoding_method == 'OneHotEncoder'):
        encoder = OneHotEncoder(sparse=False)
        target = encoder.fit_transform(df[['mbti']])

    return target

def Scaling(scale_method, X_train, X_test):
    if( scale_method == 'No Scale'):
        return X_train, X_test

    elif(scale_method == 'StandardScaler'):
        scaler = StandardScaler()

    elif(scale_method == 'MinMaxScaler'):
        scaler = MinMaxScaler()

    elif(scale_method == 'RobustScaler'):
        scaler = RobustScaler()


    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    return X_train_scaled, X_test_scaled


In [37]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (4081, 9)
Shape of y: (65296,)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
import itertools

# grid search(hyperparameter)
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# random search(hyperparameter)
param_dist = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# combinations of hyperparameters
combinations = list(itertools.product(encoder_list, scaler_list, test_size_list, do_smote))

for encoder, scaler, test_size, do_smote in combinations:
    print(f"Encoder: {encoder}, Scaler: {scaler}, Test Size: {test_size}, Smote Phase: {do_smote}")
    wandb.config.update({
        'encoder': encoder,
        'scaler': scaler,
        'test_size': test_size,
        'do_smote': do_smote
    })

    y = Encoding(df=df, encoding_method=encoder).flatten()

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=test_size)

    X_train, X_test = Scaling(scale_method=scaler, X_train=X_train, X_test=X_test)

    if do_smote == 'True':
        smote = SMOTE(sampling_strategy='auto', random_state=0)
        X_train, y_train = smote.fit_resample(X_train, y_train)

    print('Train 피처/레이블 데이터 세트:', X_train.shape, y_train.shape)

    wandb.log({'Train feature 데이터 세트': X_train.shape})
    wandb.log({'Train label 데이터 세트': X_train.shape})

    # Decision Tree Model with hyperparameter tuning
    dt_model = DecisionTreeClassifier(random_state=42)

    # Grid Search
    grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5)
    grid_search.fit(X_train, y_train)

    best_params_grid = grid_search.best_params_
    dt_model_grid = DecisionTreeClassifier(random_state=42, **best_params_grid)
    dt_model_grid.fit(X_train, y_train)
    dt_predictions_grid = dt_model_grid.predict(X_test)
    dt_accuracy_grid = accuracy_score(y_test, dt_predictions_grid)
    dt_confusion_matrix_grid = confusion_matrix(y_test, dt_predictions_grid, labels=np.unique(y))
    dt_f1_score_grid = f1_score(y_test, dt_predictions_grid, average='macro')
    dt_recall_grid = recall_score(y_test, dt_predictions_grid, average='macro')
    dt_precision_grid = precision_score(y_test, dt_predictions_grid, average='macro')
    print("Decision Tree Accuracy with Grid Search:", dt_accuracy_grid)
    print("Decision Tree Confusion Matrix with Grid Search:\n", dt_confusion_matrix_grid)
    print("Decision Tree F1 Score with Grid Search:", dt_f1_score_grid)
    print("Decision Tree Recall with Grid Search:", dt_recall_grid)
    print("Decision Tree Precision with Grid Search:", dt_precision_grid)
    wandb.log({'Decision Tree Accuracy with Grid Search': dt_accuracy_grid})
    wandb.log({'Decision Tree Confusion Matrix with Grid Search': dt_confusion_matrix_grid.tolist()})
    wandb.log({'Decision Tree F1 Score with Grid Search': dt_f1_score_grid})
    wandb.log({'Decision Tree Recall with Grid Search': dt_recall_grid})
    wandb.log({'Decision Tree Precision with Grid Search': dt_precision_grid})

    # Random Search
    random_search = RandomizedSearchCV(estimator=dt_model, param_distributions=param_dist, cv=5)
    random_search.fit(X_train, y_train)

    best_params_random = random_search.best_params_
    dt_model_random = DecisionTreeClassifier(random_state=42, **best_params_random)
    dt_model_random.fit(X_train, y_train)
    dt_predictions_random = dt_model_random.predict(X_test)
    dt_accuracy_random = accuracy_score(y_test, dt_predictions_random)
    dt_confusion_matrix_random = confusion_matrix(y_test, dt_predictions_random, labels=np.unique(y))
    dt_f1_score_random = f1_score(y_test, dt_predictions_random, average='macro')
    dt_recall_random = recall_score(y_test, dt_predictions_random, average='macro')
    dt_precision_random = precision_score(y_test, dt_predictions_random, average='macro')
    print("Decision Tree Accuracy with Random Search:", dt_accuracy_random)
    print("Decision Tree Confusion Matrix with Random Search:\n", dt_confusion_matrix_random)
    print("Decision Tree F1 Score with Random Search:", dt_f1_score_random)
    print("Decision Tree Recall with Random Search:", dt_recall_random)
    print("Decision Tree Precision with Random Search:", dt_precision_random)
    wandb.log({'Decision Tree Accuracy with Random Search': dt_accuracy_random})
    wandb.log({'Decision Tree Confusion Matrix with Random Search': dt_confusion_matrix_random.tolist()})
    wandb.log({'Decision Tree F1 Score with Random Search': dt_f1_score_random})
    wandb.log({'Decision Tree Recall with Random Search': dt_recall_random})
    wandb.log({'Decision Tree Precision with Random Search': dt_precision_random})

    # K-Fold 
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    dt_model_cv = DecisionTreeClassifier(random_state=42, **best_params_grid)
    cv_scores = cross_val_score(dt_model_cv, X_train, y_train, cv=kf, scoring='accuracy')
    cv_accuracy = np.mean(cv_scores)
    print("Decision Tree Accuracy with K-Fold Cross Validation:", cv_accuracy)
    wandb.log({'Decision Tree Accuracy with K-Fold Cross Validation': cv_accuracy})


Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.3, Smote Phase: T
Train 피처/레이블 데이터 세트: (2856, 9) (2856,)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Decision Tree Accuracy with Grid Search: 0.1983673469387755
Decision Tree Confusion Matrix with Grid Search:
 [[16 20  3  4  2  2  0  1  2  0  1  2 10 13  0  2]
 [10 30  2  5  2 12  0  2  2  0  5  2  2  8  0  1]
 [ 7 18 19 12  0  2  0  3  3  0  5  1  1  7  0  8]
 [ 8 18  3 39  0  3  0  4  1  0  3  4  0  5  0  6]
 [ 4 13  0  1  0  0  0  3  2  0  0  1  5  2  0  1]
 [ 6 16  4  7  4 11  0  7  0  0  4  2  6  1  0  0]
 [ 2 17  1  5  0  2  0  2  0  0  1  1  7  1  0  2]
 [ 3 14  9 18  0 13  0  9  0  0  8  2  4  6  0 13]
 [11  5  0  1  1  2  0  0 15  0  7  6 37 11  0  0]
 [ 6  8  1  2  3  4  0  0 10  0  6  9 25 13  0  2]
 [ 5  8  4  5  0  1  0  1  5  0 17 15  6 19  0  8]
 [10  8  3  1  1  0  0  3  3  0 12 26  9  9  0  5]
 [ 9  6  0  2  4  1  0  1  3  0  0  7 27  2  0  1]
 [ 8  9  3  0  0  3  0  0  2  0  7  4 12 20  0  2]
 [ 7  5  4  1  2  1  0  0  1  0  3 13 14  6  0  2]
 [ 6 15  7 11  1  1  0  1  2  0  9  5  5  6  0 14]]
Decision Tree F1 Score with Grid Search: 0.16181521964893003
Decision Tre

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Decision Tree Accuracy with Random Search: 0.18285714285714286
Decision Tree Confusion Matrix with Random Search:
 [[24  0 13  7  0  1  0  1  9  5  1  2  5  2  0  8]
 [25  0  8 18  0  8  1  1  5  3  0  2  2  2  1  7]
 [10  0 22 20  0  8  1  0  4  1  2  0  0  2  1 15]
 [ 8  0  7 49  0  3  0  1  4  1  2  3  0  2  0 14]
 [13  0  0  5  0  1  0  2  4  2  0  2  2  0  0  1]
 [19  0  4 15  0 12  0  3  3  1  0  1  3  2  1  4]
 [ 8  0  2  9  0  7  0  2  3  1  1  1  3  1  0  3]
 [17  0  7 26  0 10  2  4  4  2  4  1  2  1  1 18]
 [11  0  4  1  0  2  0  0 29  7  4  7 28  1  0  2]
 [13  0  2  4  0  5  1  0 20 13  5  5 16  2  0  3]
 [ 7  0  9  9  0  1  1  1 21  2  5  9  4  7  3 15]
 [12  0  4  7  0  2  2  2  7  0  7 23  7  4  1 12]
 [24  0  1  3  0  0  0  0  8  5  0  4 17  0  0  1]
 [14  0  5  4  0  3  0  0 11  2  4  1  9  6  1 10]
 [20  0  3  2  0  2  0  0  5  3  0 10  6  4  1  3]
 [13  0 13 16  0  3  0  0  2  2  6  4  1  2  2 19]]
Decision Tree F1 Score with Random Search: 0.1419192475273517
Decisi

...

Decision Tree Accuracy with Random Search: 0.193390452876377
Decision Tree Confusion Matrix with Random Search:
 [[ 7  2  6  3  1  0  0  0  1  4  0  1  5 17  0  0]
 [ 5 12  6  3  2  5  0  1  0  2  3  1  0 11  0  1]
 [ 3  4 24  8  0  0  0  1  2  0  6  0  1  5  0  2]
 [ 4  6 13 20  0  2  0  2  0  0  3  4  0  5  0  3]
 [ 5  4  1  0  1  0  0  0  0  0  0  0  3  4  0  0]
 [ 2  7  7  3  4  9  0  3  0  2  2  0  3  6  0  0]
 [ 1  7  3  3  0  2  0  1  0  4  1  0  2  1  0  2]
 [ 5  1 13 16  0  7  0  7  0  2  4  2  1  7  0  3]
 [ 2  3  1  1  1  0  0  0  8 19  4  6  9 11  0  0]
 [ 7  3  2  2  0  2  0  0  2 15  4  7  4 10  0  0]
 [ 5  1  8  1  0  1  0  0  4  4  9  9  0 14  0  3]
 [ 5  4  4  0  2  2  0  1  0  7  9 19  2  8  0  2]
 [ 5  1  0  2  2  0  0  0  1 18  0  5  4  8  0  2]
 [ 2  6  4  0  0  2  0  0  1  6  5  2  5 20  0  0]
 [ 2  2  4  0  0  0  0  0  1  5  1 10  3  9  0  1]
 [ 4  2 16  8  0  1  0  0  0  1  5  6  1  6  0  3]]
Decision Tree F1 Score with Random Search: 0.1607909474772016
Decision

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Decision Tree Accuracy with Grid Search: 0.2078239608801956
Decision Tree Confusion Matrix with Grid Search:
 [[ 3  5  2  0  0  0  0  0  1  0  1  0  3  2  0  1]
 [ 1  8  3  0  0  4  0  1  0  1  1  0  1  1  0  1]
 [ 1  4  7  6  0  0  0  2  1  0  2  2  0  2  0  2]
 [ 0  1  4  8  0  2  0  2  0  0  2  2  0  1  0  2]
 [ 2  5  1  0  0  0  0  0  0  0  0  0  2  1  0  0]
 [ 1 11  3  2  0  3  0  2  0  0  2  0  1  0  0  1]
 [ 1  4  4  1  0  0  0  1  0  0  0  1  1  0  0  2]
 [ 1  4  5 10  0  4  0  3  0  0  1  0  0  1  0  6]
 [ 0  3  1  0  0  0  0  0  4  0  4  2 14  0  0  1]
 [ 1  4  0  2  0  2  0  0  1  0  3  1 13  2  0  0]
 [ 0  1  2  1  0  0  0  0  2  1  7  3  0  6  0  4]
 [ 3  5  2  0  0  2  0  0  2  1  2 14  4  1  0  4]
 [ 2  6  0  1  0  0  0  0  1  0  1  4 13  0  0  1]
 [ 0  9  0  0  0  1  0  0  2  1  4  2  6  8  0  0]
 [ 0  4  1  0  0  0  0  0  0  0  1  6  5  3  0  0]
 [ 0  3  2  3  0  0  0  0  0  0  2  3  1  1  0  7]]
Decision Tree F1 Score with Grid Search: 0.16619637267082296
Decision Tre

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Decision Tree Accuracy with Random Search: 0.2078239608801956
Decision Tree Confusion Matrix with Random Search:
 [[ 3  5  2  0  0  0  0  0  1  0  1  0  3  2  0  1]
 [ 1  8  3  0  0  4  0  1  0  1  1  0  1  1  0  1]
 [ 1  4  7  6  0  0  0  2  1  0  2  2  0  2  0  2]
 [ 0  1  4  8  0  2  0  2  0  0  2  2  0  1  0  2]
 [ 2  5  1  0  0  0  0  0  0  0  0  0  2  1  0  0]
 [ 1 11  3  2  0  3  0  2  0  0  2  0  1  0  0  1]
 [ 1  4  4  1  0  0  0  1  0  0  0  1  1  0  0  2]
 [ 1  4  5 10  0  4  0  3  0  0  1  0  0  1  0  6]
 [ 0  3  1  0  0  0  0  0  4  0  4  2 14  0  0  1]
 [ 1  4  0  2  0  2  0  0  1  0  3  1 13  2  0  0]
 [ 0  1  2  1  0  0  0  0  2  1  7  3  0  6  0  4]
 [ 3  5  2  0  0  2  0  0  2  1  2 14  4  1  0  4]
 [ 2  6  0  1  0  0  0  0  1  0  1  4 13  0  0  1]
 [ 0  9  0  0  0  1  0  0  2  1  4  2  6  8  0  0]
 [ 0  4  1  0  0  0  0  0  0  0  1  6  5  3  0  0]
 [ 0  3  2  3  0  0  0  0  0  0  2  3  1  1  0  7]]
Decision Tree F1 Score with Random Search: 0.16619637267082296
Decisi

In [ ]:
wandb.finish()